# <font color=orange><div align="center">SDP Project - Question 4</div></font>

### <font color=orange><div align="center">Janvier 2026</div></font>
### <font color=orange><div align="center">Ouissal BOUTOUATOU - Alae TAOUDI - Mohammed SBAIHI</div></font>


Refer to [Problem Formulation](Question4.md) for rigorous mathematical formulation of the problem

In [13]:
from gurobipy import *

## Instanciation

In [2]:
CANDIDATES = {
    "x": [85, 81, 71, 69, 75, 81, 88],
    "y": [81, 81, 75, 63, 67, 88, 95],
    "z": [74, 89, 74, 81, 68, 84, 79],
    "t": [74, 71, 84, 91, 77, 76, 73],
    "u": [72, 75, 66, 85, 88, 66, 93],
    "v": [71, 73, 63, 92, 76, 79, 93],
    "w": [79, 69, 78, 76, 67, 84, 79],
    "w'": [57, 76, 81, 76, 82, 86, 77],
    "a1": [89, 74, 81, 68, 84, 79, 77],
    "a2": [71, 84, 91, 79, 78, 73.5, 77]
}

WEIGHTS = [8, 7, 7, 6, 6, 5, 6]
COURSES = ["A", "B", "C", "D", "E", "F", "G"]

In [3]:
def pros_and_cons(x, y):
    """Returns the pros(x,y) and cons(x, y) mappings: Course -> Contribution to x > y
    Inputs: x, y students
    N.B: courses are annotated using integers instead of alphabets
    """
    assert x in CANDIDATES, f"{x} isn't in candidates"
    assert y in CANDIDATES, f"{y} isn't in candidates"

    pros = {
        i: (CANDIDATES[x][i] - CANDIDATES[y][i]) * WEIGHTS[i]
        for i in range(len(CANDIDATES[x]))
        if (CANDIDATES[x][i] - CANDIDATES[y][i]) > 0
    }

    cons = {
        i: (CANDIDATES[x][i] - CANDIDATES[y][i]) * WEIGHTS[i]
        for i in range(len(CANDIDATES[x]))
        if (CANDIDATES[x][i] - CANDIDATES[y][i]) < 0
    }
    return pros, cons

## Partie 1 : Démonstration de la non-existence d'une explication (1-m) pour z > t

In [4]:
# Analyse pour z > t
PROS_ZT, CONS_ZT = pros_and_cons("z", "t")
print("Pros(z, t):", PROS_ZT)
print("Cons(z, t):", CONS_ZT)
print("\nContributions détaillées:")
for course_idx, contrib in PROS_ZT.items():
    print(f"  {COURSES[course_idx]}: +{contrib}")
for course_idx, contrib in CONS_ZT.items():
    print(f"  {COURSES[course_idx]}: {contrib}")

Pros(z, t): {1: 126, 5: 40, 6: 36}
Cons(z, t): {2: -70, 3: -60, 4: -54}

Contributions détaillées:
  B: +126
  F: +40
  G: +36
  C: -70
  D: -60
  E: -54


### Vérification exhaustive : Trade-offs (1-m) possibles

In [5]:
from itertools import combinations

print("Test de tous les trade-offs (1-m) possibles:\n")
print("="*80)

valid_1m_tradeoffs = {}

# Pour chaque cours pro, tester tous les sous-ensembles de cons
for p in PROS_ZT:
    pros_course = COURSES[p]
    pros_contrib = PROS_ZT[p]
    
    print(f"\nCours pro {pros_course} (+{pros_contrib}):")
    
    valid_for_this_pros = []
    
    # Tester tous les sous-ensembles non vides de cons
    for r in range(1, len(CONS_ZT) + 1):
        for cons_subset in combinations(CONS_ZT.keys(), r):
            cons_contrib = sum(CONS_ZT[c] for c in cons_subset)
            total = pros_contrib + cons_contrib
            
            cons_names = [COURSES[c] for c in cons_subset]
            
            if total > 0:
                cons_list = ', '.join(cons_names)
                print(f" [OK] Avec {{{cons_list}}}: {pros_contrib} + ({cons_contrib}) = {total} > 0")
                valid_for_this_pros.append(cons_subset)
            else:
                cons_list = ', '.join(cons_names)
                print(f" [X] Avec {{{cons_list}}}: {pros_contrib} + ({cons_contrib}) = {total} ≤ 0")
    
    valid_1m_tradeoffs[p] = valid_for_this_pros

print("\n" + "="*80)
print("RÉSUMÉ DES TRADE-OFFS (1-m) VALIDES:")
print("="*80)

for p, tradeoffs in valid_1m_tradeoffs.items():
    pros_course = COURSES[p]
    if tradeoffs:
        print(f"\nPros {pros_course} peut former {len(tradeoffs)} trade-off(s) valide(s):")
        for cons_subset in tradeoffs:
            cons_names = [COURSES[c] for c in cons_subset]
            cons_list = ', '.join(cons_names)
            print(f"  - ({pros_course}, {{{cons_list}}})")
    else:
        print(f"\nPros {pros_course}: AUCUN trade-off valide")

print("\n" + "="*80)
print("CONCLUSION: Aucune partition disjointe ne couvre tous les cons avec des (1-m) valides.")
print("="*80)

Test de tous les trade-offs (1-m) possibles:


Cours pro B (+126):
 [OK] Avec {C}: 126 + (-70) = 56 > 0
 [OK] Avec {D}: 126 + (-60) = 66 > 0
 [OK] Avec {E}: 126 + (-54) = 72 > 0
 [X] Avec {C, D}: 126 + (-130) = -4 ≤ 0
 [OK] Avec {C, E}: 126 + (-124) = 2 > 0
 [OK] Avec {D, E}: 126 + (-114) = 12 > 0
 [X] Avec {C, D, E}: 126 + (-184) = -58 ≤ 0

Cours pro F (+40):
 [X] Avec {C}: 40 + (-70) = -30 ≤ 0
 [X] Avec {D}: 40 + (-60) = -20 ≤ 0
 [X] Avec {E}: 40 + (-54) = -14 ≤ 0
 [X] Avec {C, D}: 40 + (-130) = -90 ≤ 0
 [X] Avec {C, E}: 40 + (-124) = -84 ≤ 0
 [X] Avec {D, E}: 40 + (-114) = -74 ≤ 0
 [X] Avec {C, D, E}: 40 + (-184) = -144 ≤ 0

Cours pro G (+36):
 [X] Avec {C}: 36 + (-70) = -34 ≤ 0
 [X] Avec {D}: 36 + (-60) = -24 ≤ 0
 [X] Avec {E}: 36 + (-54) = -18 ≤ 0
 [X] Avec {C, D}: 36 + (-130) = -94 ≤ 0
 [X] Avec {C, E}: 36 + (-124) = -88 ≤ 0
 [X] Avec {D, E}: 36 + (-114) = -78 ≤ 0
 [X] Avec {C, D, E}: 36 + (-184) = -148 ≤ 0

RÉSUMÉ DES TRADE-OFFS (1-m) VALIDES:

Pros B peut former 5 trade-off(s) 

## Partie 2 : Démonstration de la non-existence d'une explication (m-1) pour z > t

### Vérification exhaustive : Trade-offs (m-1) possibles

In [6]:
print("Test de tous les trade-offs (m-1) possibles pour z > t:\n")
print("="*80)

valid_m1_tradeoffs = {}

# Pour chaque cours con, tester toutes les combinaisons de pros
for c in CONS_ZT:
    cons_course = COURSES[c]
    cons_contrib = CONS_ZT[c]
    
    print(f"\nCours con {cons_course} ({cons_contrib}):")
    
    valid_for_this_cons = []
    
    # Tester toutes les combinaisons de pros (de 1 à tous)
    for r in range(1, len(PROS_ZT) + 1):
        for pros_subset in combinations(PROS_ZT.keys(), r):
            pros_contrib = sum(PROS_ZT[p] for p in pros_subset)
            total = pros_contrib + cons_contrib
            
            pros_names = [COURSES[p] for p in pros_subset]
            
            if total > 0:
                pros_list = ', '.join(pros_names)
                print(f"  [OK] Avec {{{pros_list}}}: ({pros_contrib}) + {cons_contrib} = {total} > 0")
                valid_for_this_cons.append(pros_subset)
            else:
                pros_list = ', '.join(pros_names)
                print(f"  [X] Avec {{{pros_list}}}: ({pros_contrib}) + {cons_contrib} = {total} ≤ 0")
    
    valid_m1_tradeoffs[c] = valid_for_this_cons

print("\n" + "="*80)
print("RÉSUMÉ DES TRADE-OFFS (m-1) VALIDES:")
print("="*80)

for c, tradeoffs in valid_m1_tradeoffs.items():
    cons_course = COURSES[c]
    if tradeoffs:
        print(f"\nCons {cons_course} peut être couvert par {len(tradeoffs)} combinaison(s) de pros:")
        for pros_subset in tradeoffs:
            pros_names = [COURSES[p] for p in pros_subset]
            pros_list = ', '.join(pros_names)
            print(f"  - ({{{pros_list}}}, {cons_course})")
    else:
        print(f"\nCons {cons_course}: AUCUNE combinaison de pros ne fonctionne.")

print("\n" + "="*80)
print("CONCLUSION: Ressources insuffisantes pour couvrir les 3 cons avec des (m-1) disjoints.")
print("="*80)

Test de tous les trade-offs (m-1) possibles pour z > t:


Cours con C (-70):
  [OK] Avec {B}: (126) + -70 = 56 > 0
  [X] Avec {F}: (40) + -70 = -30 ≤ 0
  [X] Avec {G}: (36) + -70 = -34 ≤ 0
  [OK] Avec {B, F}: (166) + -70 = 96 > 0
  [OK] Avec {B, G}: (162) + -70 = 92 > 0
  [OK] Avec {F, G}: (76) + -70 = 6 > 0
  [OK] Avec {B, F, G}: (202) + -70 = 132 > 0

Cours con D (-60):
  [OK] Avec {B}: (126) + -60 = 66 > 0
  [X] Avec {F}: (40) + -60 = -20 ≤ 0
  [X] Avec {G}: (36) + -60 = -24 ≤ 0
  [OK] Avec {B, F}: (166) + -60 = 106 > 0
  [OK] Avec {B, G}: (162) + -60 = 102 > 0
  [OK] Avec {F, G}: (76) + -60 = 16 > 0
  [OK] Avec {B, F, G}: (202) + -60 = 142 > 0

Cours con E (-54):
  [OK] Avec {B}: (126) + -54 = 72 > 0
  [X] Avec {F}: (40) + -54 = -14 ≤ 0
  [X] Avec {G}: (36) + -54 = -18 ≤ 0
  [OK] Avec {B, F}: (166) + -54 = 112 > 0
  [OK] Avec {B, G}: (162) + -54 = 108 > 0
  [OK] Avec {F, G}: (76) + -54 = 22 > 0
  [OK] Avec {B, F, G}: (202) + -54 = 148 > 0

RÉSUMÉ DES TRADE-OFFS (m-1) VALIDES:

Cons

## Partie 3 : Programme linéaire pour explications hybrides

### Fonction de calcul de Big-M dynamique

In [7]:
def compute_dynamic_bigM(pros, cons):
    """
    Calcule un Big-M adapté aux données pour éviter les problèmes numériques.
    Big-M = somme de toutes les valeurs absolues des cons + 1
    """
    total_negative = sum(abs(c) for c in cons.values())
    return total_negative + 1

### Construction du modèle hybride amélioré

In [8]:
def build_hybrid_model(pros, cons, verbose=True):
    """
    Construit le modèle linéaire pour explications hybrides (1-m) et (m-1).
    Retourne le modèle et toutes les variables de décision.
    """
    m = Model("Hybrid_Explanation")
    if not verbose:
        m.setParam('OutputFlag', 0)
    
    # Calcul de Big-M optimal
    BIG_M = compute_dynamic_bigM(pros, cons)
    EPSILON = 0.01
    
    if verbose:
        print(f"Big-M calculé dynamiquement: {BIG_M}")
    
    # ========== Variables de décision ==========
    
    # Variables d'assignation
    VarAssignPro = {(p, c): m.addVar(vtype=GRB.BINARY, name=f'atp_{p}_{c}')
                    for p in pros for c in cons}
    VarAssignCon = {(p, c): m.addVar(vtype=GRB.BINARY, name=f'atc_{p}_{c}')
                    for p in pros for c in cons}
    
    # Variables de pivot
    VarPivotPro = {p: m.addVar(vtype=GRB.BINARY, name=f'piv_p_{p}') for p in pros}
    VarPivotCon = {c: m.addVar(vtype=GRB.BINARY, name=f'piv_c_{c}') for c in cons}
    
    # ========== Contraintes ==========
    
    # 1. Couverture complète des cons
    for c in cons:
        m.addConstr(
            VarPivotCon[c] + quicksum([VarAssignPro[(p, c)] for p in pros]) == 1,
            name=f"coverage_con_{c}"
        )
    
    # 2. Utilisation unique des pros
    for p in pros:
        m.addConstr(
            VarPivotPro[p] + quicksum([VarAssignCon[(p, c)] for c in cons]) <= 1,
            name=f"usage_pro_{p}"
        )
    
    # 3. Contraintes de liaison
    for p in pros:
        for c in cons:
            m.addConstr(VarAssignPro[(p, c)] <= VarPivotPro[p], name=f"link_atp_{p}_{c}")
            m.addConstr(VarAssignCon[(p, c)] <= VarPivotCon[c], name=f"link_atc_{p}_{c}")
    
    # 4. Participation minimale (amélioration)
    for p in pros:
        m.addConstr(
            quicksum([VarAssignPro[(p, c)] for c in cons]) >= VarPivotPro[p],
            name=f"min_participation_pro_{p}"
        )
    
    for c in cons:
        m.addConstr(
            quicksum([VarAssignCon[(p, c)] for p in pros]) >= VarPivotCon[c],
            name=f"min_participation_con_{c}"
        )
    
    # 5. Validité des trade-offs (1-m) avec Big-M
    for p in pros:
        m.addConstr(
            pros[p] + quicksum([VarAssignPro[(p, c)] * cons[c] for c in cons])
            >= VarPivotPro[p] * EPSILON,
            name=f"validity_1m_{p}"
        )
    
    # 6. Validité des trade-offs (m-1) avec Big-M
    for c in cons:
        m.addConstr(
            cons[c] + quicksum([VarAssignCon[(p, c)] * pros[p] for p in pros])
            >= VarPivotCon[c] * EPSILON - (1 - VarPivotCon[c]) * BIG_M,
            name=f"validity_m1_{c}"
        )
    
    # ========== Fonction objectif ==========
    obj = quicksum(VarPivotPro[p] for p in pros) + quicksum(VarPivotCon[c] for c in cons)
    m.setObjective(obj, GRB.MINIMIZE)
    
    return m, VarAssignPro, VarAssignCon, VarPivotPro, VarPivotCon

### Fonction d'affichage des résultats

In [16]:
def display_hybrid_results(m, pros, cons, VarAssignPro, VarAssignCon, VarPivotPro, VarPivotCon):
    """
    Affiche les résultats du modèle hybride de manière formatée.
    """
    if m.status == GRB.OPTIMAL:
        print("="*80)
        print("SOLUTION OPTIMALE TROUVÉE !")
        print("="*80)
        print(f"\nNombre de trade-offs: {int(m.objVal)}")
        
        # Affichage des trade-offs (1-m)
        print("\n" + "="*80)
        print("TRADE-OFFS DE TYPE (1-m):")
        print("="*80)
        
        found_1m = False
        for p in pros:
            if VarPivotPro[p].X > 0.5:
                found_1m = True
                associated_cons = [c for c in cons if VarAssignPro[(p, c)].X > 0.5]
                total_contrib = pros[p] + sum(cons[c] for c in associated_cons)
                pros_name = COURSES[p]
                cons_names = [COURSES[c] for c in associated_cons]
                
                print(f"\nTrade-off: ({pros_name}, {{{', '.join(cons_names)}}})")  
                print(f"  - Contribution pro [{pros_name}]: {pros[p]:+d}")
                for c in associated_cons:
                    print(f"  - Contribution con [{COURSES[c]}]: {cons[c]:+d}")
                print(f"  - Total: {total_contrib:+.2f} > 0")
        
        if not found_1m:
            print("\nAucun trade-off (1-m)")
        
        # Affichage des trade-offs (m-1)
        print("\n" + "="*80)
        print("TRADE-OFFS DE TYPE (m-1):")
        print("="*80)
        
        found_m1 = False
        for c in cons:
            if VarPivotCon[c].X > 0.5:
                found_m1 = True
                associated_pros = [p for p in pros if VarAssignCon[(p, c)].X > 0.5]
                total_contrib = cons[c] + sum(pros[p] for p in associated_pros)
                cons_name = COURSES[c]
                pros_names = [COURSES[p] for p in associated_pros]
                
                print(f"\nTrade-off: ({{{', '.join(pros_names)}}}, {cons_name})")
                for p in associated_pros:
                    print(f"  - Contribution pro [{COURSES[p]}]: {pros[p]:+d}")
                print(f"  - Contribution con [{cons_name}]: {cons[c]:+d}")
                print(f"  - Total: {total_contrib:+.2f} > 0")
        
        if not found_m1:
            print("\nAucun trade-off (m-1)")
        
        # Vérification finale
        print("\n" + "="*80)
        print("VÉRIFICATION:")
        print("="*80)
        
        covered_cons = set()
        for (p, c) in VarAssignPro:
            if VarAssignPro[(p, c)].X > 0.5:
                covered_cons.add(c)
        for c in cons:
            if VarPivotCon[c].X > 0.5:
                covered_cons.add(c)
        
        cons_to_cover = set(cons.keys())
        all_covered = covered_cons == cons_to_cover
        
        print(f"Tous les cons couverts: {all_covered} {'[OK]' if all_covered else '[X]'}")
        print(f"Cons couverts: {[COURSES[c] for c in sorted(covered_cons)]}")
        
    elif m.status == GRB.INFEASIBLE:
        print("="*80)
        print("MODÈLE INFAISABLE !")
        print("="*80)
        print("\nAucune explication hybride n'existe pour cette comparaison.")
        print("Certificat de non-existence établi par le solveur.")
    else:
        print(f"\nStatut du solveur: {m.status}")

## Résolution pour z > t

In [17]:
print("="*80)
print("RÉSOLUTION POUR z > t")
print("="*80)

PROS_zt, CONS_zt = pros_and_cons("z", "t")
print(f"\nPros: {PROS_zt}")
print(f"Cons: {CONS_zt}")
print()

RÉSOLUTION POUR z > t

Pros: {1: 126, 5: 40, 6: 36}
Cons: {2: -70, 3: -60, 4: -54}



In [18]:
m_zt, atp_zt, atc_zt, piv_p_zt, piv_c_zt = build_hybrid_model(PROS_zt, CONS_zt, verbose=True)
m_zt.optimize()

Big-M calculé dynamiquement: 185
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 11+.0 (26200.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1360P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Academic license 2754426 - for non-commercial use only - registered to ou___@student-cs.fr
Optimize a model with 36 rows, 24 columns and 108 nonzeros (Min)
Model fingerprint: 0x3c77f2d7
Model has 6 linear objective coefficients
Variable types: 0 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [1e-02, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 2.0000000
Presolve removed 36 rows and 24 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution cou

In [19]:
display_hybrid_results(m_zt, PROS_zt, CONS_zt, atp_zt, atc_zt, piv_p_zt, piv_c_zt)

SOLUTION OPTIMALE TROUVÉE !

Nombre de trade-offs: 2

TRADE-OFFS DE TYPE (1-m):

Trade-off: (B, {D, E})
  - Contribution pro [B]: +126
  - Contribution con [D]: -60
  - Contribution con [E]: -54
  - Total: +12.00 > 0

TRADE-OFFS DE TYPE (m-1):

Trade-off: ({F, G}, C)
  - Contribution pro [F]: +40
  - Contribution pro [G]: +36
  - Contribution con [C]: -70
  - Total: +6.00 > 0

VÉRIFICATION:
Tous les cons couverts: True [OK]
Cons couverts: ['C', 'D', 'E']


## Partie 4 : Application aux candidats a1 et a2

In [20]:
print("="*80)
print("RÉSOLUTION POUR a1 > a2")
print("="*80)

PROS_a1a2, CONS_a1a2 = pros_and_cons("a1", "a2")
print(f"\nPros(a1, a2): {PROS_a1a2}")
print(f"Cons(a1, a2): {CONS_a1a2}")
print("\nContributions détaillées:")
for course_idx, contrib in PROS_a1a2.items():
    print(f"  {COURSES[course_idx]}: +{contrib}")
for course_idx, contrib in CONS_a1a2.items():
    print(f"  {COURSES[course_idx]}: {contrib}")
print()

RÉSOLUTION POUR a1 > a2

Pros(a1, a2): {0: 144, 4: 36, 5: 27.5}
Cons(a1, a2): {1: -70, 2: -70, 3: -66}

Contributions détaillées:
  A: +144
  E: +36
  F: +27.5
  B: -70
  C: -70
  D: -66



In [21]:
m_a1a2, atp_a1a2, atc_a1a2, piv_p_a1a2, piv_c_a1a2 = build_hybrid_model(PROS_a1a2, CONS_a1a2, verbose=True)
m_a1a2.optimize()

Big-M calculé dynamiquement: 207
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 11+.0 (26200.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1360P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Academic license 2754426 - for non-commercial use only - registered to ou___@student-cs.fr
Optimize a model with 36 rows, 24 columns and 108 nonzeros (Min)
Model fingerprint: 0x1ad7696c
Model has 6 linear objective coefficients
Variable types: 0 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [1e-02, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 34 rows and 24 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [22]:
display_hybrid_results(m_a1a2, PROS_a1a2, CONS_a1a2, atp_a1a2, atc_a1a2, piv_p_a1a2, piv_c_a1a2)

MODÈLE INFAISABLE !

Aucune explication hybride n'existe pour cette comparaison.
Certificat de non-existence établi par le solveur.


## Interprétation des résultats pour a1 > a2

Le modèle est **infaisable** pour la comparaison $a_1 \succ a_2$.

### Signification

Même avec le modèle hybride (le plus flexible), le solveur retourne un statut **INFEASIBLE**. Ceci constitue un **certificat formel de non-existence** : il est mathématiquement impossible de partitionner les pros et cons de $a_1$ en un ensemble de trade-offs disjoints (1-m) ou (m-1) qui couvrent tous les cons de $a_2$.

### Analyse

Bien que $a_1$ ait une moyenne pondérée supérieure à $a_2$, cette supériorité **ne peut pas être expliquée** par des arguments structurés de type (1-m) ou (m-1). Voici pourquoi :

1. **Avantage principal consommé** : L'avantage majeur de $a_1$ en Anatomie (A: +144) est presque entièrement utilisé pour compenser les deux désavantages en Biologie (B: -70) et Diagnostic (D: -66).

2. **Avantages résiduels insuffisants** : Les avantages restants en Épidémiologie (E: +36) et Forensique (F: +27.5) ne sont pas assez forts pour former des trade-offs valides avec les contraintes de disjonction.

3. **Impossibilité structurelle** : Il n'existe aucune façon de partitionner les ressources (pros) pour couvrir tous les désavantages (cons) tout en maintenant des trade-offs strictement positifs et disjoints.

### Conclusion

La supériorité de $a_1$ sur $a_2$ résulte d'un **équilibre global complexe** (trade-off de type m-n général) qui dépasse la logique des explications structurées (1-m) et (m-1). C'est un exemple où la préférence agrégée existe mais ne peut être décomposée en arguments simples.